<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>

# Week 2: Identify Nearest Health Facilities

<span style="color:red">
**UPDATE**

Thank you for your analysis. Despite our warning efforts so far, the virus continues to spread rapidly. We want to get infected individuals treatment as quickly as possible, so we need your help to calculate which hospital or clinic is closest to each known infected individual in the population.
</span>

Your goal for this notebook will be to identify the nearest hospital or clinic for each infected person.

## Imports

In [1]:
%load_ext cudf.pandas
import pandas as pd
import cuml
import cupy as cp

## Load Population Data

Begin by loading the `lat`, `long` and `infected` columns from `'./data/week2.csv'` into a cuDF-accelerated pandas DataFrame called `df`.

In [2]:
df = pd.read_csv("./data/week2.csv")
df

,lat,long,infected
0,54.522510,-1.571896,0.0
1,54.554030,-1.524968,0.0
2,54.552486,-1.435203,0.0
3,54.537189,-1.566215,0.0
4,54.528212,-1.588462,0.0
...,...,...,...
58479889,51.634416,-2.925863,0.0
58479890,51.556972,-3.036290,0.0
58479891,51.588992,-2.921915,0.0
58479892,51.590974,-2.954539,0.0


## Load Hospital and Clinics Data

For this step, your goal is to create an `all_med` cuDF-accelearted pandas DataFrame that contains the latitudes and longitudes of all the hospitals (data found at `'./data/hospitals.csv'`) and clinics (data found at `'./data/clinics.csv'`).

In [5]:
hospitals_df = pd.read_csv("./data/hospitals.csv")
hospitals_df

,OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.37999725,-0.406042069,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.31513214,-0.556289494,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.43719482,-2.847192764,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.4597435,-2.245468855,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.07812119,-0.030604055,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,10617466,RTH18,Hospital,UNKNOWN,Independent Sector,Visible,False,University Hospital Of Wales,C/O John Radcliffe Hospital,Headley Way,...,<NA>,OX3 9DU,51.76387405,-1.219792008,RTH,Oxford University Hospitals NHS Foundation Trust,<NA>,<NA>,<NA>,<NA>
1225,10617482,RX3KI,Hospital,UNKNOWN,Independent Sector,Visible,False,Foss Park Hospital,Huntington House,Jockey Lane,...,<NA>,YO32 9XW,53.98666382,-1.051122308,RX3,"Tees, Esk and Wear Valleys NHS Foundation Trust",<NA>,<NA>,<NA>,<NA>
1226,10617567,RGT1W,Hospital,UNKNOWN,Independent Sector,Visible,False,Jersey General Hospital,The Parade,<NA>,...,<NA>,JE1 3UH,<NA>,<NA>,RGT,Cambridge University Hospitals NHS Foundation ...,<NA>,<NA>,<NA>,<NA>
1227,10617714,RAX0A,Hospital,UNKNOWN,Independent Sector,Visible,False,London North West University Hospitals,Watford Road,<NA>,...,<NA>,HA1 3UJ,51.57540894,-0.322022736,RAX,Kingston Hospital NHS Foundation Trust,<NA>,<NA>,<NA>,<NA>


In [4]:
clinics_df = pd.read_csv("./data/clinics.csv")
clinics_df

,OrganisationID,OrganisationCode,OrganisationType,SubType,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,Address3,City,County,Postcode,Latitude,Longitude,Phone,Email,Website,Fax
0,12366,F81157,Clinic,UNKNOWN,Visible,True,Frinton Road Medical Centre,<NA>,"68 Frinton Road, Hollond-on-Sea",<NA>,Clacton-on-Sea,<NA>,CO15 5UW,51.804237,1.186376,<NA>,<NA>,<NA>,<NA>
1,12450,F81670,Clinic,UNKNOWN,Visible,True,Epping Close Surgery,<NA>,Epping Close,<NA>,Clacton on Sea,<NA>,CO15 4UZ,51.815262,1.154707,01255 222668,<NA>,<NA>,<NA>
2,12461,F81681,Clinic,UNKNOWN,Visible,True,Green Elms Medical Centre,<NA>,32 Crossways,<NA>,"Jaywick, Clacton on Sea",<NA>,CO15 2NB,51.780621,1.117907,<NA>,<NA>,<NA>,<NA>
3,17937,NAA01,Clinic,UNKNOWN,Visible,False,St Chads Clinic,St. Chads Drive,<NA>,<NA>,Liverpool,Merseyside,L32 8RE,53.482368,-2.885404,<NA>,<NA>,<NA>,<NA>
4,17938,NAA02,Clinic,UNKNOWN,Visible,False,Whiston Health Centre,Old Colliery Road,Whiston,<NA>,Prescot,Merseyside,L35 3SX,53.415630,-2.800874,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19077,10617742,NBG7L,Clinic,UNKNOWN,Visible,False,Sunnybank Medical Centre,Town Gate,Wyke,<NA>,Bradford,<NA>,BD12 9NG,53.737923,-1.767982,<NA>,<NA>,<NA>,<NA>
19078,10617743,NFH02,Clinic,UNKNOWN,Visible,False,Somerset Surgical Services At Mendip Vale Medi...,Pudding Pie Lane,Langford,<NA>,Bristol,<NA>,BS40 5EL,51.341579,-2.784808,<NA>,<NA>,<NA>,<NA>
19079,10617744,NFH03,Clinic,UNKNOWN,Visible,False,Somerset Surgical Services At Axbridge Medical...,Houlgate Way,<NA>,<NA>,Axbridge,<NA>,BS26 2BJ,51.286285,-2.820388,<NA>,<NA>,<NA>,<NA>
19080,10617746,NEMAM,Clinic,UNKNOWN,Visible,False,Lawton House Surgery,Bromley Road,<NA>,<NA>,Congleton,<NA>,CW12 1QG,53.163048,-2.205781,<NA>,<NA>,<NA>,<NA>


In [6]:
all_med = pd.concat([hospitals_df, clinics_df])
all_med

,OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.459743,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19077,10617742,NBG7L,Clinic,UNKNOWN,<NA>,Visible,False,Sunnybank Medical Centre,Town Gate,Wyke,...,<NA>,BD12 9NG,53.737923,-1.767982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19078,10617743,NFH02,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Mendip Vale Medi...,Pudding Pie Lane,Langford,...,<NA>,BS40 5EL,51.341579,-2.784808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19079,10617744,NFH03,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Axbridge Medical...,Houlgate Way,<NA>,...,<NA>,BS26 2BJ,51.286285,-2.820388,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
19080,10617746,NEMAM,Clinic,UNKNOWN,<NA>,Visible,False,Lawton House Surgery,Bromley Road,<NA>,...,<NA>,CW12 1QG,53.163048,-2.205781,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


Since we will be using the coordinates of those facilities, keep only those rows that are non-null in both  `Latitude` and `Longitude`.

**Tip**: Reset the index of `all_med` with `.reset_index(drop=True)`. 

In [7]:
all_med = all_med.dropna(subset=["Latitude","Longitude"])

all_med = all_med.reset_index(drop=True)

all_med 

,OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,County,Postcode,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,Surrey,KT12 3LD,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,Surrey,GU22 7HS,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,Avon,BS21 6BS,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,Greater Manchester,M15 5AT,53.459743,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,<NA>,SG8 5JP,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20296,10617742,NBG7L,Clinic,UNKNOWN,<NA>,Visible,False,Sunnybank Medical Centre,Town Gate,Wyke,...,<NA>,BD12 9NG,53.737923,-1.767982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
20297,10617743,NFH02,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Mendip Vale Medi...,Pudding Pie Lane,Langford,...,<NA>,BS40 5EL,51.341579,-2.784808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
20298,10617744,NFH03,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Axbridge Medical...,Houlgate Way,<NA>,...,<NA>,BS26 2BJ,51.286285,-2.820388,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
20299,10617746,NEMAM,Clinic,UNKNOWN,<NA>,Visible,False,Lawton House Surgery,Bromley Road,<NA>,...,<NA>,CW12 1QG,53.163048,-2.205781,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Make Grid Coordinates for Medical Facilities

Provided for you in the next cell (which you can expand by clicking on the "...", and contract again after executing by clicking on the blue left border of the cell) is the lat/long to grid coordinates converter you have used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `all_med` DataFrame you created in the last step.

In [8]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [9]:
cupy_lat = cp.asarray(all_med["Latitude"])
cupy_long = cp.asarray(all_med["Longitude"])

all_med['northing'], all_med['easting'] = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
all_med

,OrganisationID,OrganisationCode,OrganisationType,SubType,Sector,OrganisationStatus,IsPimsManaged,OrganisationName,Address1,Address2,...,Latitude,Longitude,ParentODSCode,ParentName,Phone,Email,Website,Fax,northing,easting
0,17970,NDA07,Hospital,Hospital,Independent Sector,Visible,True,Walton Community Hospital - Virgin Care Servic...,<NA>,Rodney Road,...,51.379997,-0.406042,NDA,Virgin Care Services Ltd,01932 414205,<NA>,<NA>,01932 253674,165810.473974,510917.517174
1,17981,NDA18,Hospital,Hospital,Independent Sector,Visible,True,Woking Community Hospital (Virgin Care),<NA>,Heathside Road,...,51.315132,-0.556289,NDA,Virgin Care Services Ltd,01483 715911,<NA>,<NA>,<NA>,158381.343420,500604.836652
2,18102,NLT02,Hospital,Hospital,NHS Sector,Visible,True,North Somerset Community Hospital,North Somerset Community Hospital,Old Street,...,51.437195,-2.847193,NLT,North Somerset Community Partnership Community...,01275 872212,<NA>,http://www.nscphealth.co.uk,<NA>,171305.775859,341119.365090
3,18138,NMP01,Hospital,Hospital,Independent Sector,Visible,False,Bridgewater Hospital,120 Princess Road,<NA>,...,53.459743,-2.245469,NMP,Bridgewater Hospital (Manchester) Ltd,0161 2270000,<NA>,www.bridgewaterhospital.com,<NA>,395944.561405,383703.600293
4,18142,NMV01,Hospital,Hospital,Independent Sector,Visible,True,Kneesworth House,Old North Road,Bassingbourn,...,52.078121,-0.030604,NMV,Partnerships In Care Ltd,01763 255 700,reception_kneesworthhouse@partnershipsincare.c...,www.partnershipsincare.co.uk,<NA>,244071.710013,534945.182860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20296,10617742,NBG7L,Clinic,UNKNOWN,<NA>,Visible,False,Sunnybank Medical Centre,Town Gate,Wyke,...,53.737923,-1.767982,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,426887.040596,415302.519662
20297,10617743,NFH02,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Mendip Vale Medi...,Pudding Pie Lane,Langford,...,51.341579,-2.784808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,160625.127012,345341.262026
20298,10617744,NFH03,Clinic,UNKNOWN,<NA>,Visible,False,Somerset Surgical Services At Axbridge Medical...,Houlgate Way,<NA>,...,51.286285,-2.820388,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,154503.665383,342794.591249
20299,10617746,NEMAM,Clinic,UNKNOWN,<NA>,Visible,False,Lawton House Surgery,Bromley Road,<NA>,...,53.163048,-2.205781,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,362932.494686,386243.408863


## Find Closest Hospital or Clinic for Infected

Fit `cuml.NearestNeighbors` with `all_med`'s `northing` and `easting` values, using the named argument `n_neighbors` set to `1`, and save the model as `knn`.

In [11]:
knn = cuml.NearestNeighbors(n_neighbors=1)
knn.fit(all_med[["northing", "easting"]])

NearestNeighbors()

Save every infected member in `df` into a new dataframe called `infected_df`.

**Tip**: Reset the index of `infected_df` with `.reset_index(drop=True)`. 

In [12]:
infected_filter = df['infected']==True
infected_df = df[infected_filter]
infected_df.reset_index(drop=True)

,lat,long,infected
0,53.715826,-2.430079,1.0
1,53.664881,-2.425673,1.0
2,53.696765,-2.488940,1.0
3,53.696966,-2.488897,1.0
4,53.727804,-2.392959,1.0
...,...,...,...
70875,51.662717,-2.926850,1.0
70876,51.599350,-2.959175,1.0
70877,51.543825,-2.822984,1.0
70878,51.562536,-2.879492,1.0


Create `northing` and `easting` values for `infected_df`.

In [13]:
cupy_infected_lat = cp.asarray(infected_df["lat"])
cupy_infected_long = cp.asarray(infected_df["long"])

infected_df['northing'], infected_df['easting'] = latlong2osgbgrid_cupy(cupy_infected_lat,cupy_infected_long)
infected_df

,lat,long,infected,northing,easting
1346586,53.715826,-2.430079,1.0,424489.783814,371619.678741
1350932,53.664881,-2.425673,1.0,418820.687944,371876.492369
1352085,53.696765,-2.488940,1.0,422394.398940,367721.000265
1352799,53.696966,-2.488897,1.0,422416.821887,367723.973098
1357529,53.727804,-2.392959,1.0,425808.109929,374076.557677
...,...,...,...,...,...
58459189,51.662717,-2.926850,1.0,196451.131712,335900.971171
58459551,51.599350,-2.959175,1.0,189433.449408,333573.002864
58461935,51.543825,-2.822984,1.0,183143.984735,342935.130365
58464082,51.562536,-2.879492,1.0,185270.296666,339042.062616


Use `knn.kneighbors` with `n_neighbors=1` on `infected_df`'s `northing` and `easting` values. Save the return values in `distances` and `indices`.

In [14]:
distances, indices = knn.kneighbors(infected_df[["northing", "easting"]])
infected_df['distance']=distances
infected_df['closest_clinic_hospital']=indices
infected_df

,lat,long,infected,northing,easting,distance,closest_clinic_hospital
1346586,53.715826,-2.430079,1.0,424489.783814,371619.678741,2562.830078,18316
1350932,53.664881,-2.425673,1.0,418820.687944,371876.492369,2988.618164,12816
1352085,53.696765,-2.488940,1.0,422394.398940,367721.000265,1162.685059,7785
1352799,53.696966,-2.488897,1.0,422416.821887,367723.973098,1162.283203,7785
1357529,53.727804,-2.392959,1.0,425808.109929,374076.557677,1455.115601,4962
...,...,...,...,...,...,...,...
58459189,51.662717,-2.926850,1.0,196451.131712,335900.971171,8617.324219,10015
58459551,51.599350,-2.959175,1.0,189433.449408,333573.002864,1306.979248,10015
58461935,51.543825,-2.822984,1.0,183143.984735,342935.130365,7633.295410,3814
58464082,51.562536,-2.879492,1.0,185270.296666,339042.062616,7071.362305,10015


### View Closest Clinic/Hospital ###

`indices`, returned from your use of `knn.kneighbors` immediately above, should map person indices to their closest clinic/hospital indices:

Here you can print an infected individual's coordinates from `infected_df`:

In [15]:
infected_df.iloc[0] # get the coords of an infected individual (in this case, individual 0)

lat                            53.715826
long                           -2.430079
infected                        1.000000
northing                   424489.783814
easting                    371619.678741
distance                     2562.830078
closest_clinic_hospital     18316.000000
Name: 1346586, dtype: float64

You should be able to used the mapped index for the nearest facility to see that indeed the nearest facility is at a nearby coordinate:

In [16]:
all_med.iloc[18316] # printing the entry for facility 1234 (replace with the index identified as closest to the individual)

OrganisationID                                             9445963
OrganisationCode                                             NPR0F
OrganisationType                                            Clinic
SubType                                                    UNKNOWN
Sector                                                        None
OrganisationStatus                                         Visible
IsPimsManaged                                                False
OrganisationName      Community Dermatology Service - About Health
Address1                                     The Innovation Centre
Address2                                          1 Evolution Park
Address3                                           Haslingden Road
City                                                     Blackburn
County                                                  Lancashire
Postcode                                                   BB1 2FD
Latitude                                                 53.73

## New Patients ##

Use the `knn` model to find the closest clinic/hospital for the below new patients. 

In [17]:
# DO NOT CHANGE THIS CELL
patients=[
    {
        "name":"John Smith", 
        "northing":462643.2282915547,
        "easting":363859.7580565622
    }, 
        {
        "name":"Greg Brown", 
        "northing":409324.1030472915,
        "easting":464084.5085059871
    }
]

patients_df=pd.DataFrame(patients)

patients_df['distance'], patients_df['closest_clinic_hospital'] = knn.kneighbors(patients_df[['northing', 'easting']])

patients_df['OrganisationID'] = patients_df.apply(lambda x: all_med.loc[x['closest_clinic_hospital'], "OrganisationID"], axis=1)

patients_df

,name,northing,easting,distance,closest_clinic_hospital,OrganisationID
0,John Smith,462643.228292,363859.758057,7286.685059,4637,64351
1,Greg Brown,409324.103047,464084.508506,1561.915405,3886,62389


In [18]:
patients_df.to_json('my_assessment/question_2.json', orient='records')

/opt/conda/lib/python3.10/site-packages/cudf/io/json.py:194: UserWarning: Using CPU via Pandas to write JSON dataset
  warnings.warn("Using CPU via Pandas to write JSON dataset")


## Check Submission ##

In [19]:
!cat my_assessment/question_2.json

[{"name":"John Smith","northing":462643.2282915547,"easting":363859.7580565622,"distance":7286.6850585938,"closest_clinic_hospital":4637,"OrganisationID":64351},{"name":"Greg Brown","northing":409324.1030472915,"easting":464084.5085059871,"distance":1561.9154052734,"closest_clinic_hospital":3886,"OrganisationID":62389}]

**Tip**: Your submission file should contain one line of text, similar to: 

```
[{'name':'John Smith','northing':XXX.XX,'easting':XXX.XX,'closest_clinic_hospital':XX,'OrganisationID':XX},{'name':'Greg Brown','northing':XXX.XX,'easting':XXX.XX,'closest_clinic_hospital':XX,'OrganisationID':XX}]
```

<div align="center"><h2>Please Restart the Kernel</h2></div>

...before moving to the next notebook.

In [20]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>